In [1]:
using Test
using MTCGP
import YAML

@testset "MTCGPInd construction" begin
    cfg = get_config("../MTCGP/cfg/test.yaml")
    ind = MTCGPInd(cfg)

    @test length(ind.nodes) == 3 * 10 + 4
    for node in ind.nodes
        if node.active
            @test node.x >= 1
            @test node.x <= length(ind.nodes)
            @test node.y >= 1
            @test node.y <= length(ind.nodes)
        end
    end
end

@testset "Processing" begin
    cfg = YAML.load_file("../MTCGP/cfg/test.yaml")
    cfg["functions"] = ["f_abs", "f_add", "f_mult"]
    cfg = get_config(cfg)
    ind = MTCGPInd(cfg)

    inputs = zeros(4)
    set_inputs(ind, inputs)
    for i in 1:4
        @test ind.buffer[i] == 0.0
    end
    output = process(ind)
    @test output[1] == 0.0
    for i in eachindex(ind.nodes)
        if ind.nodes[i].active
            @test ind.buffer[i] == 0.0
        end
    end

    output = process(ind, ones(4))
    @test output[1] == 1.0
    for i in eachindex(ind.nodes)
        if ind.nodes[i].active
            @test ind.buffer[i] == 1.0
        end
    end

    output = process(ind, Array{MType}([ones(3) for i in 1:4]))
    @test all(output[1] .== ones(3))
    for i in eachindex(ind.nodes)
        if ind.nodes[i].active
            @test all(ind.buffer[i] == ones(3))
        end
    end
end

Test Summary:         | Pass  Total
MTCGPInd construction |    9      9
Test Summary: | Pass  Total
Processing    |   13     13


Test.DefaultTestSet("Processing", Any[], 13, false)

In [1]:
using PyCall
vzd = pyimport("vizdoom")
random = pyimport("random")

PyObject <module 'random' from 'C:\\Users\\benny\\.julia\\conda\\3\\lib\\random.py'>

In [4]:
py"""
import itertools as it
def act(game, a_id):
    actions = [list(a) for a in it.product([0, 1], repeat=5)]
    return game.make_action(actions[a_id])
"""

game = vzd.DoomGame()
game.set_doom_scenario_path("../scenarios/basic.wad")
    # Sets map to start (scenario .wad files can contain many maps).
    game.set_doom_map("map01")

    # Sets resolution. Default is 320X240
    game.set_screen_resolution(vzd.ScreenResolution.RES_640X480)

    # Sets the screen buffer format. Not used here but now you can change it. Defalut is CRCGCB.
    game.set_screen_format(vzd.ScreenFormat.RGB24)

    # Enables depth buffer.
    game.set_depth_buffer_enabled(true)

    # Enables labeling of in game objects labeling.
    game.set_labels_buffer_enabled(true)

    # Enables buffer with top down map of the current episode/level.
    game.set_automap_buffer_enabled(true)

    # Sets other rendering options (all of these options except crosshair are enabled (set to True) by default)
    game.set_render_hud(false)
    game.set_render_minimal_hud(false)  # If hud is enabled
    game.set_render_crosshair(false)
    game.set_render_weapon(true)
    game.set_render_decals(false)  # Bullet holes and blood on the walls
    game.set_render_particles(false)
    game.set_render_effects_sprites(false)  # Smoke and blood
    game.set_render_messages(false)  # In-game messages
    game.set_render_corpses(false)
    game.set_render_screen_flashes(true)  # Effect upon taking damage or picking up items

    # Adds buttons that will be allowed.
    game.add_available_button(vzd.Button.MOVE_LEFT)
    game.add_available_button(vzd.Button.MOVE_RIGHT)
    game.add_available_button(vzd.Button.MOVE_FORWARD)
    game.add_available_button(vzd.Button.MOVE_BACKWARD)
    game.add_available_button(vzd.Button.ATTACK)

    # Adds game variables that will be included in state.
    game.add_available_game_variable(vzd.GameVariable.AMMO2)

    # Causes episodes to finish after 200 tics (actions)
    game.set_episode_timeout(200)

    # Makes episodes start after 10 tics (~after raising the weapon)
    game.set_episode_start_time(10)

    # Makes the window appear (turned on by default)
    game.set_window_visible(true)

    # Turns on the sound. (turned off by default)
    game.set_sound_enabled(true)

    # Sets the livin reward (for each move) to -1
    game.set_living_reward(-1)

    # Sets ViZDoom mode (PLAYER, ASYNC_PLAYER, SPECTATOR, ASYNC_SPECTATOR, PLAYER mode is default)
    game.set_mode(vzd.Mode.PLAYER)

    # Enables engine output to console.
    #game.set_console_enabled(True)

    # Initialize the game. Further configuration won't take any effect from now on.
    game.init()

    # Define some actions. Each list entry corresponds to declared buttons:
    # MOVE_LEFT, MOVE_RIGHT, ATTACK
    # game.get_available_buttons_size() can be used to check the number of available buttons.
    # 5 more combinations are naturally possible but only 3 are included for transparency when watching.
    actions = [[true, false, false, false, false], [false, true, false, false, false], [false, false, true, false, false],
               [false, false, false, true, false],  [false, false, false, false, true]]

    # Run this many episodes
    episodes = 10

    # Sets time that will pause the engine after each action (in seconds)
    # Without this everything would go too fast for you to keep track of what's happening.
    sleep_time = 0

    py"""
    import itertools as it
    def act(game, a_id):
        actions = [list(a) for a in it.product([0, 1], repeat=5)]
        return game.make_action(actions[a_id])
    """

     for i in length(episodes)
        print("Episode #" * string(i))

        # Starts a new episode. It is not needed right after init() but it doesn't cost much. At least the loop is nicer.
        game.new_episode()

        while game.is_episode_finished()!=true

            # Gets the state
            state = game.get_state()

            # Which consists of:
            n = state.number
            vars = state.game_variables
            screen_buf = state.screen_buffer
            depth_buf = state.depth_buffer
            labels_buf = state.labels_buffer
            automap_buf = state.automap_buffer
            labels = state.labels

            # Games variables can be also accessed via:
            #game.get_game_variable(GameVariable.AMMO2)

            # Makes a random action and get remember reward.
            r = py"act"(game, random.choice(actions))
            # Makes a "prolonged" action and skip frames:
            # skiprate = 4
            # r = game.make_action(choice(actions), skiprate)

            # The same could be achieved with:
            # game.set_action(choice(actions))
            # game.advance_action(skiprate)
            # r = game.get_last_reward()

            # Prints state's game variables and reward.
            print("State #" + str(n))
            print("Game variables:", vars)
            print("Reward:", r)
            print("=====================")

            if sleep_time > 0
                sleep(sleep_time)
            end

        # Check how the episode went.
        print("Episode finished.")
        print("Total reward:", game.get_total_reward())
        print("************************")
            
        end

    # It will be done automatically anyway but sometimes you need to do it in the middle of the program...
    game.close()
end

Episode #1

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\benny\.julia\packages\PyCall\zqDXB\src\pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'TypeError'>
TypeError('only integer scalar arrays can be converted to a scalar index')
  File "C:\Users\benny\.julia\packages\PyCall\zqDXB\src\pyeval.jl", line 4, in act
